In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps 'xformers<0.0.27' "trl<0.9.0" peft accelerate bitsandbytes evaluate

In [ ]:
#Download data from github repository and extract
!git clone https://github.com/ryanzhumich/AESLC.git


Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461
Receiving objects: 100% (17469/17469), 7.36 MiB | 9.56 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Unsloth: Will load unsloth/mistral-7b-v0.3-bnb-4bit as a legacy tokenizer.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from google.colab import userdata
import huggingface_hub
huggingface_hub.login(userdata.get('HuggingFace'))
#from huggingface_hub import notebook_login
#notebook_login()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#Generate JSON file from email dataset
import os
import json
import pandas as pd

# Define the folder containing the text files
folder_path = '/content/AESLC/enron_subject_line/train'

# Initialize lists to store the data
data = []
instruction = 'Please help summarize the provided email body and generate email subject'
# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".subject"):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            # Split the content into body and subject
            if '@subject' in content:
                body_text, subject_text = content.split('@subject')
                data.append({
                    'instruction': instruction,
                    'input': body_text.strip(),
                    'output': subject_text.strip()
                })

# Save the data to a JSON file
json_path = '/content/dataset.json'
with open(json_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print(f"JSON file saved to {json_path}")

JSON file saved to /content/dataset.json


In [ ]:
email_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = email_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import Dataset
import json
# Load your custom dataset
json_path = '/content/dataset.json'

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
#from datasets import load_dataset
#dataset = load_dataset("/content/dataset.json", split = "train")
dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/14436 [00:00<?, ? examples/s]

In [ ]:
dataset_dict = dataset.train_test_split(test_size=0.005)

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_WATCH=all
%env WANDB_SILENT=true

env: WANDB_WATCH=all
env: WANDB_SILENT=true


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=a6eeca43a4d7ed509dbcee617b7c8f2d0e939287fed4d87218811079b9f3c0c4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install evaluate
import evaluate
rouge = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Move predictions and labels to CPU
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = [[(label if label != -100 else tokenizer.pad_token_id) for label in label_seq] for label_seq in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return result



<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers.utils import logging
#import wandb

#logging.set_verbosity_info()
#project_name = "mistral7b-unsloth"
#entity = "wandb"
## os.environ["WANDB_LOG_MODEL"] = "checkpoint"

#wandb.init(project=project_name, name = "mistral7b-unsloth-sft")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps = 4,
        evaluation_strategy="steps",
        warmup_steps = 5,
        num_train_epochs=3,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
  #      report_to="wandb",  # enable logging to W&B
        logging_strategy = 'steps',
       # save_total_limit=2,
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/14363 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/73 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.52 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()
#wandb.finish()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 14,363 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,2.638700,2.662342
2,2.785500,2.492314
3,2.318500,2.324615


Step,Training Loss,Validation Loss
1,2.638700,2.662342
2,2.785500,2.492314
3,2.318500,2.324615
4,2.272200,2.166678
5,2.095200,2.041522
6,2.151800,1.910907
7,1.963400,1.843856
8,1.702700,1.798777
9,1.726000,1.771891
10,1.835700,1.760116


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2152.5146 seconds used for training.
35.88 minutes used for training.
Peak reserved memory = 7.738 GB.
Peak reserved memory for training = 3.218 GB.
Peak reserved memory % of max memory = 52.468 %.
Peak reserved memory for training % of max memory = 21.82 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# email_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    email_prompt.format(
        "Please help summarize the provided email body and generate email subject", # instruction
        "Phillip,   Could you please do me a favor?I would like  to read your current title policy to see what it says about easements.You  should have received a copy during your closing.I don't know how many  pages it will be but let me know how you want to handle getting a copy  made.I'll be happy to make the copy, or whatever makes it easy for  you.Thanks,", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPlease help summarize the provided email body and generate email subject\n\n### Input:\nPhillip,   Could you please do me a favor?I would like  to read your current title policy to see what it says about easements.You  should have received a copy during your closing.I don't know how many  pages it will be but let me know how you want to handle getting a copy  made.I'll be happy to make the copy, or whatever makes it easy for  you.Thanks,\n\n### Response:\nTitle Policy</s>"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# email_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    email_prompt.format(
        "Please help summarize the provided email body and generate email subject", # instruction
        "Phillip,   Could you please do me a favor?I would like  to read your current title policy to see what it says about easements.You  should have received a copy during your closing.I don't know how many  pages it will be but let me know how you want to handle getting a copy  made.I'll be happy to make the copy, or whatever makes it easy for  you.Thanks,", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please help summarize the provided email body and generate email subject

### Input:
Phillip,   Could you please do me a favor?I would like  to read your current title policy to see what it says about easements.You  should have received a copy during your closing.I don't know how many  pages it will be but let me know how you want to handle getting a copy  made.I'll be happy to make the copy, or whatever makes it easy for  you.Thanks,

### Response:
Title Policy</s>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("EmailSubGen_lora_model") # Local saving
tokenizer.save_pretrained("EmailSubGen_lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('EmailSubGen_lora_model/tokenizer_config.json',
 'EmailSubGen_lora_model/special_tokens_map.json',
 'EmailSubGen_lora_model/tokenizer.model',
 'EmailSubGen_lora_model/added_tokens.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
!unzip EmailSubGen_lora_model.zip

Archive:  EmailSubGen_lora_model.zip
   creating: content/EmailSubGen_lora_model/
  inflating: content/EmailSubGen_lora_model/special_tokens_map.json  
  inflating: content/EmailSubGen_lora_model/adapter_model.safetensors  
  inflating: content/EmailSubGen_lora_model/tokenizer.model  
  inflating: content/EmailSubGen_lora_model/tokenizer_config.json  
  inflating: content/EmailSubGen_lora_model/adapter_config.json  
  inflating: content/EmailSubGen_lora_model/README.md  


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "content/EmailSubGen_lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    email_prompt.format(
        "Please help summarize the provided email body and generate email subject", # instruction
        "The following reports have been waiting for your approval for more than 4 days.Please review.Owner: James W Reitmeyer Report Name: JReitmeyer 10/24/01 Days In Mgr.Queue: 5", # input
        "", # output - leave this blank for generation!
    ),
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth: Fast Mistral patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: Will load unsloth/mistral-7b-v0.3-bnb-4bit as a legacy tokenizer.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPlease help summarize the provided email body and generate email subject\n\n### Input:\nThe following reports have been waiting for your approval for more than 4 days.Please review.Owner: James W Reitmeyer Report Name: JReitmeyer 10/24/01 Days In Mgr.Queue: 5\n\n### Response:\nReports Waiting for Your Approval</s>']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
import os
import json
import pandas as pd

# Define the folder containing the text files
folder_path = '/content/AESLC/enron_subject_line/test'

# Initialize lists to store the data
data = []
instruction = 'Please help summarize the provided email body and generate email subject'
# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".subject"):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            # Split the content into body and subject
            if '@subject' in content:
                body_text, subject_text = content.split('@subject')

                lines = subject_text.strip().splitlines()  # Split by lines and remove leading/trailing whitespace
                output = []
                for line in lines:
                    if line.strip():
                        if line.startswith("@"):
                            annotation = line.split()[1:]  # Extract annotation text after "@" and split by space
                            if len(annotation):
                                output.append("".join(annotation))  # Join words in annotation back together
                        else:
                            output.append(line.strip())  # Add subject or remaining text after removing whitespace
                data.append({
                    'instruction': instruction,
                    'input': body_text.strip(),
                    'output': output
                })

# Save the data to a JSON file
json_path = '/content/AESLC/enron_subject_line/test/testdataset.json'
with open(json_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print(f"JSON file saved to {json_path}")


JSON file saved to /content/AESLC/enron_subject_line/test/testdataset.json


In [ ]:
email_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = email_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import Dataset
import json
# Load your custom dataset
json_path = '/content/AESLC/enron_subject_line/test/testdataset.json'

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

testdataset = Dataset.from_list(data[:100])
testdataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/14436 [00:00<?, ? examples/s]

In [ ]:
!pip install rouge

In [ ]:
from transformers import pipeline
from evaluate import load
from rouge_score import rouge_scorer

# Load the ROUGE metric
rouge = load("rouge")

# Create a text generation pipeline
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

all_scores = []
# Use index-based access for Dataset objects
for i in range(len(testdataset[:1000])):
    example = testdataset[i] # Access element like a list
    input_text = example['input']
    references = example['output']  # List of references

    # Generate prediction for the input
    prediction = generator(input_text, max_new_tokens=100, num_beams=1)[0]['generated_text']

    # Calculate ROUGE scores against each reference and store the best
    best_scores = {'rouge1': {'fmeasure': 0}, 'rouge2': {'fmeasure': 0}, 'rougeL': {'fmeasure': 0}}
    for ref in references:
        scores = scorer.score(prediction, ref)
        for metric, score in scores.items():
            if score.fmeasure > best_scores[metric]['fmeasure']:
                best_scores[metric] = {'fmeasure': score.fmeasure, 'precision': score.precision, 'recall': score.recall}

    all_scores.append(best_scores)

# Aggregate scores (e.g., calculate average)
avg_scores = {
    'rouge1': {'fmeasure': 0, 'precision': 0, 'recall': 0},
    'rouge2': {'fmeasure': 0, 'precision': 0, 'recall': 0},
    'rougeL': {'fmeasure': 0, 'precision': 0, 'recall': 0}
}

for scores in all_scores:
    for metric in avg_scores:
        for key in ['fmeasure', 'precision', 'recall']:
            avg_scores[metric][key] += scores[metric][key]

for metric in avg_scores:
    for key in ['fmeasure', 'precision', 'recall']:
        avg_scores[metric][key] /= len(all_scores)

# Print the average ROUGE scores
print(avg_scores)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

KeyError: 'precision'

In [ ]:

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

all_scores = []
# Use index-based access for Dataset objects
for i in range(len(testdataset[:1000])):
    example = testdataset[i] # Access element like a list
    input_text = example['input']
    references = example['output']  # List of references

    # Generate prediction for the input
    prediction = generator(input_text, max_new_tokens=100, num_beams=1)[0]['generated_text']

    # Calculate ROUGE scores against each reference and store the best
    best_scores = {'rouge1': {'fmeasure': 0}, 'rouge2': {'fmeasure': 0}, 'rougeL': {'fmeasure': 0}}
    for ref in references:
        scores = scorer.score(prediction, ref)
        for metric, score in scores.items():
            if score.fmeasure > best_scores[metric]['fmeasure']:
                best_scores[metric] = {'fmeasure': score.fmeasure, 'precision': score.precision, 'recall': score.recall}

    all_scores.append(best_scores)

# Aggregate scores (e.g., calculate average)
avg_scores = {
    'rouge1': {'fmeasure': 0, 'precision': 0, 'recall': 0},
    'rouge2': {'fmeasure': 0, 'precision': 0, 'recall': 0},
    'rougeL': {'fmeasure': 0, 'precision': 0, 'recall': 0}
}

for scores in all_scores:
    for metric in avg_scores:
        for key in ['fmeasure', 'precision', 'recall']:
            avg_scores[metric][key] += scores[metric][key]

for metric in avg_scores:
    for key in ['fmeasure', 'precision', 'recall']:
        avg_scores[metric][key] /= len(all_scores)

# Print the average ROUGE scores
print(avg_scores)

NameError: name 'generator' is not defined

In [ ]:
from transformers import pipeline
from evaluate import load

# Load the ROUGE metric
rouge = load("rouge")

# Create a text generation pipeline
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# Generate predictions on the test dataset
predictions = generator(
    [example["input"] for example in testdataset[:100]],
    max_new_tokens=100,
    num_beams=1,
)

# Extract the generated text from the pipeline output
predictions = [pred[0]['generated_text'] for pred in predictions]

# Compute ROUGE metrics
results = rouge.compute(predictions=predictions, references=testdataset["output"])

# Print the results
print(results)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

TypeError: string indices must be integers

In [ ]:
from transformers import pipeline
from evaluate import load

# Load the ROUGE metric
rouge = load("rouge")

# Create a text generation pipeline
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# Generate predictions on the test dataset
# Access the input column of the dataset using the column name
predictions = generator(
    testdataset[:100]["input"],
    max_new_tokens=100,
    num_beams=1,
)

# Extract the generated text from the pipeline output
predictions = [pred[0]['generated_text'] for pred in predictions]

# Compute ROUGE metrics
results = rouge.compute(predictions=predictions, references=testdataset[:100]["output"])

# Print the results
print(results)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

{'rouge1': 0.04175057546404236, 'rouge2': 0.015307029349338995, 'rougeL': 0.03865576026979294, 'rougeLsum': 0.040112317820734385}


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

In [ ]:
# prompt: zip the folder and download zip folder in colab

!zip -r /content/EmailSubGen_lora_model.zip /content/EmailSubGen_lora_model
from google.colab import files
files.download("/content/EmailSubGen_lora_model.zip")


  adding: content/EmailSubGen_lora_model/ (stored 0%)
  adding: content/EmailSubGen_lora_model/special_tokens_map.json (deflated 77%)
  adding: content/EmailSubGen_lora_model/adapter_model.safetensors (deflated 7%)
  adding: content/EmailSubGen_lora_model/tokenizer.model (deflated 61%)
  adding: content/EmailSubGen_lora_model/tokenizer_config.json (deflated 96%)
  adding: content/EmailSubGen_lora_model/adapter_config.json (deflated 54%)
  adding: content/EmailSubGen_lora_model/README.md (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/model/unsloth.Q4_K_M.gguf.zip /content/model/unsloth.Q4_K_M.gguf